# Computing the spectra to test the window function

Eu tentei gerar o catálogo de randoms como uma função de seleção, mas não dá certo. Está faltando alguma coisa. Lembrando que o catálogo de randoms está mascarado e o catálogo normal não.

### 0) Including the libraries

In [1]:
from catalog_specs import cat_specs #to set the catalog specifications
from cosmo import cosmo #to set the cosmology
from code_options import code_parameters #to set the cosmological parameters
from converting_cats_in_cats_of_bins import * #To convert any halo catalog according to bins of mass
from colossus.cosmology import cosmology #To compute bias
from colossus.lss import bias #To compute bias
from create_grids_from_xyz_cats_beta import * #to build the cats
from MTPK_estimate_beta import * #to estimate the spectra
import numpy as np
import matplotlib.pyplot as plt

### 1) Instantiating the parameters

The fiducial **specifications** about the **catalogs** are given by:

In [2]:
cat_specs = cat_specs(ntracers = 1, n_maps = 1, 
                      x_cat_min = -500.1, y_cat_min = -500.1, z_cat_min = 4995.0,
                     x_cat_max = 500.1, y_cat_max = 500.1, z_cat_max = 6001.0)
cat_specs.parameters_print()

ntracers = 1
n_maps = 1
col_m = 6
col_x = 0
col_y = 1
col_z = 2
x_cat_min = -500.1
y_cat_min = -500.1
z_cat_min = 4995.0
x_cat_max = 500.1
y_cat_max = 500.1
z_cat_max = 6001.0
m_min = 11.5
m_max = 13.0


The **cosmology** is given by:

In [3]:
h = 0.6711
zmin = 4.0 #Minimum redshift
zmax = 7.8 #Maximum redshift
zcentral = (zmin + zmax)/2. #Mean redshift in the data
my_cosmology = cosmo(h = h, Omega0_b = 0.049, Omega0_cdm = 0.2685,
              Omega0_k = 0.0, Omega0_DE = 0.6825, A_s = 2.1277e-9,
              n_s = 0.9624, w0 = - 1.0, w1 = 0.0, z_re = 9.99999,
              flat = True, gamma = 0.5454, zcentral = zcentral,
                    matgrow = cosmo().f_evolving(0))

In [4]:
my_cosmology.parameters_print()

h = 0.6711
Omega0_b = 0.049
Omega0_cdm = 0.2685
Omega0_k = 0.0
Omega0_DE = 0.6825
A_s = 2.1277e-09
ln10e10AsA = 3.057626677091581
n_s = 0.9624
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrow = 0.5252511834620611
zcentral = 5.9
c_light = 299792.458


### 2) Catalogs

In [5]:
MF = np.array([44812/(1000.**3)])
MF

array([4.4812e-05])

In [6]:
b = [1.0]
b

[1.0]

### 3) Setting `MTPK code` options

Using the computed _mass function_ and _bias_

In [7]:
#Defining the code options
my_code_options = code_parameters(verbose = True, method = 'FKP',
                                  mas_method = 'CIC',
                                  multipoles_order = 4,
                                  do_cross_spectra = False, 
                                  ntracers = 1,
                                  mass_fun = MF,
                                  cell_size = 4.0,
                                  n_x = 250, n_y = 250, n_z = 250,
                                  n_x_orig = -125.0, n_y_orig = -125.0, n_z_orig = 5000.0,
                                  use_kmin_phys = False, kmin_phys = 0.025,
                                  use_kmax_phys = True, kmax_phys = 0.5,
                                  tracer_bias = np.array(b),
                                  sel_fun_data = True,
                                  sel_fun_file = 'sel_fun.hdf5')

In [8]:
my_code_options.parameters_print()

verbose = True
method = FKP
mas_method = CIC
ntracers = 1
use_kdip_phys = False
kdip_phys = 0.005
multipoles_order = 4
do_cross_spectra = False
use_padding = False
padding_length = [10, 10, 10]
use_theory_spectrum = False
theory_spectrum_file = theory_spectrum_file.txt
use_mask = False
mask_filename = mask.hdf5
mass_fun = [4.4812e-05]
tracer_bias = [1.]
cell_size = 4.0
n_x = 250
n_y = 250
n_z = 250
n_x_orig = -125.0
n_y_orig = -125.0
n_z_orig = 5000.0
sel_fun_data = True
sel_fun_file = sel_fun.hdf5
kmin_bias = 0.05
kmax_bias = 0.15
kph_central = 0.1
dkph_bin = 0.01
use_kmin_phys = False
kmin_phys = 0.025
use_kmax_phys = True
kmax_phys = 0.5
whichspec = 1
mult_sel_fun = 1.0
shift_sel_fun = 0.0
k_min_CAMB = 0.0001
k_max_CAMB = 1.0
use_redshifts = False
mask_redshift = False
save_mask = False
save_mean_sel_fun = False
split_tracers = False
tracer_bins = [3.16e+11 1.00e+12 3.16e+12 1.00e+13]
mask_spillover_cells = False
batch_size = 1000000
wrap = False


### 4) Creating the random catalog

You can create the cats using `create_grids_from_xyz_cats_beta`

In [9]:
#User definitions about where to do each thing
input_filename = 'window'
filenames_catalogs = 'data/window/cutted-random_RSD_xyz-L0-5000.dat'
dir_out = "maps/data/window/"

In [ ]:
create_grids_from_xyz_cats(cat_specs, my_cosmology, my_code_options, 
                           input_filename, filenames_catalogs, dir_out)


Will load maps stored in files:
[['data/window/cutted-random_RSD_xyz-L0-5000.dat']]
Dimensions of the grids: n_x, n_y, n_z = 250 250 250

The actual catalog spans the ranges in x,y,z:
x: -500.1 --> 500.1
y: -500.1 --> 500.1
z: 4995.0 --> 6001.0


With the padding length, of  0 cells, the box will be filled with:
x: ( 0 * 0 , 250 , 0 *0)
y: ( 0 * 0 , 250 , 0 *0)
z: ( 0 * 0 , 250 , 0 *0)

Check: given the padding, your catalog should end at cartesian positions:
max(x) = 1000.0
max(y) = 1000.0
max(z) = 1000.0

Origin (0,0,0) of box will be considered to be displaced from the observer @Earth
by these numbers of cells in each direction:    (This affects RSDs!)
n_x_orig= -125.0
n_y_orig= -125.0
n_z_orig= 5000.0


Mass assignement: Clouds in Cell (CiC)

Processing catalog # 0
Reading catalog for tracer 0
Original catalog has 9698230 objects
Processing batch # 0 of 9
Processing objects # 0 to 1000000


### 5) Estimating the spectra

You can estimate the spectra using the function inside `MTPK_estimate_beta` code. The `handle_data` is the same as the `input_file_name`

In [ ]:
#Some user options
handle_data = "window"
dir_maps = 'maps/sims/window/'
dir_data = 'maps/data/window/'
dir_specs = 'spectra/window/'

In [ ]:
MTPK_estimate(cat_specs, my_cosmology, my_code_options, dir_maps, dir_data, dir_specs, handle_data = handle_data)

### 6) Some plots

In [ ]:
# Taking data
P0_FKP = np.loadtxt('spectra/window/window_P0_FKP.dat')

P2_FKP = np.loadtxt('spectra/window/window_P2_FKP.dat')

P4_FKP = np.loadtxt('spectra/window/window_P4_FKP.dat')

k = np.loadtxt('spectra/window/window_vec_k.dat')

In [ ]:
n_bins = cat_specs.ntracers
n_cats = cat_specs.n_maps
n_k = k.shape[0]

P0_FKP = P0_FKP.reshape(n_bins, n_cats, n_k)

P2_FKP = P2_FKP.reshape(n_bins, n_cats, n_k)

P4_FKP = P4_FKP.reshape(n_bins, n_cats, n_k)

In [ ]:
import theory as theory

In [ ]:
Pk_dict = theory.power_spectrum(my_cosmology, my_code_options)

In [ ]:
Pk_dict.keys()

In [ ]:
rsd_pars = theory.rsd_params(b1 = np.asarray([zcentral]),
                             sigma_tot = np.asarray([300])) #Redshiftspace distortion parameters

In [ ]:
M_dict = theory.pk_multipoles_gauss(rsd_pars, my_cosmology, my_code_options, 
                                    Nk = 463, k = Pk_dict['k'])

In [ ]:
redshift = np.asarray([zcentral])

In [ ]:
P_mono = Pk_dict['Pk_'+str(redshift[0])]*M_dict['mono'][0][0]
P_mono.shape

In [ ]:
P0_FKP.shape, k.shape

In [ ]:
plt.figure(dpi = 100)
for i in range(n_bins):
    plt.loglog(k, np.mean(P0_FKP, axis = 1)[i]/3., label = f'FKP Monopole - Bin {i}')
    plt.loglog(Pk_dict['k'], P_mono, color = 'b', linestyle = ':', linewidth = 0.8, label = '$P_0$')
    plt.legend()

In [ ]:
plt.figure(dpi = 100)
for i in range(n_bins):
    plt.plot(k, np.mean(P2_FKP, axis = 1)[i], label = f'FKP Dipole - Bin {i}')
    plt.legend()

In [ ]:
plt.figure(dpi = 100)
for i in range(n_bins):
    plt.plot(k, np.mean(P4_FKP, axis = 1)[i], label = f'FKP Quadrupole - Bin {i}')
    plt.legend()